In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install ivis

     |████████████████████████████████| 655kB 5.0MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391581 sha256=6d766cf26109d47c6587a6d98776838d5504ddc289771ee70bba2c299ffe6af8
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import roc_auc_score, matthews_corrcoef
from sklearn.multiclass import OneVsRestClassifier

import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("Data/input.csv") #pd.read_csv("/gdrive/My Drive/Colab_Notebooks/Sentence-Analysis/input.csv")
#test_df = pd.read_csv("/gdrive/My Drive/Colab_Notebooks/Sentence-Analysis/input_test.csv")

#mappings = {1: 0, 2: 1, 3:2}

#train_df.label = [mappings[item] for item in train_df.label]
train_df

,Unnamed: 0,patient_id,sequence,label
0,0,Z8399761,"(unclear history), venous insufficiency, PVD, ...",2
1,1,Z11957588,period. It apparently cuts car accident rates ...,0
2,2,Z10809276,(P) 5 Hearing: no concerns Vision: needs updat...,1
3,3,Z8539944,"remission, s/p 1 suicide attempt, 3 IPLOC admi...",2
4,4,Z12696803,should be pursued at that point in time. He an...,0
...,...,...,...,...
1295,1295,Z7648831,"including memory, attention/executive function...",2
1296,1296,Z7701777,"minimal stimuli · Psychiatry in to see, in not...",2
1297,1297,Z7648831,"including memory, attention/executive function...",2
1298,1298,Z7648831,"including memory, attention/executive function...",2


In [13]:
X = train_df["sequence"]
y = train_df["label"]

y_label = y.to_numpy()
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X,y,random_state=0,test_size=0.1, stratify=y_label)

y_train_valid_label = y_train_valid.to_numpy()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=0, test_size=0.1, stratify=y_train_valid_label)

In [16]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words="english",analyzer='word')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)
count_valid = count_vectorizer.transform(X_valid)

# Create the CountVectorizer DataFrame: count_train
count_train = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the CountVectorizer DataFrame: test_df
count_test = pd.DataFrame(count_test.A, columns=count_vectorizer.get_feature_names())

count_valid = pd.DataFrame(count_valid.A, columns=count_vectorizer.get_feature_names())
count_valid.head()

,00,01,0148,02,02114,04,05,06,07,08,...,yau,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english",analyzer='word')
tfidf_train= tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
tfidf_valid = tfidf_vectorizer.transform(X_valid)

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_train  = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_test  = pd.DataFrame(tfidf_test.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_valid  = pd.DataFrame(tfidf_valid.A, columns=tfidf_vectorizer.get_feature_names())

tfidf_valid.head()

,00,01,0148,02,02114,04,05,06,07,08,...,yau,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
def filter_features_by_cor(df):
    m = len(df.columns)
    output = df.iloc[:,m-1] 
    output_list = output.tolist()
    corrcoef_array = []

    for i in range(0,m-2):
        input_list = df.iloc[:,i].tolist()
        cols = [input_list, output_list]
        corrcoef = abs(np.corrcoef(cols)) 
        #print(type(corrcoef))
        corrcoef_array = np.append(corrcoef_array,corrcoef[0,1])

    feature_names = list(df)
    feature_names = feature_names[0:m-2]
    output_df = pd.DataFrame(feature_names, columns=['Features'])
    output_df['CorrCoef'] = corrcoef_array
    output_df = output_df.sort_values('CorrCoef')
    output_df = output_df.reset_index()
    output_df = output_df.drop(columns = "index")
    
    return output_df

In [83]:
tfidf_features_df = pd.concat([tfidf_train, y_train.reset_index(drop=True)], axis=1)
tfidf_features_df.head()

,00,01,0148,02,02114,04,05,06,07,08,...,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial,label
0,0.000000,0.402627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.320483,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [86]:
tfidf_output_df = filter_features_by_cor(tfidf_features_df)
tfidf_output_df = tfidf_output_df.sort_values(by=['CorrCoef'],ascending = False)
tfidf_output_df.to_csv('Data/tfidf_vector_feature_corr.csv')

tfidf_output_df

,Features,CorrCoef
1166,mother,0.624384
1165,dementia,0.623368
1164,mild,0.595557
1163,history,0.504810
1162,cognitive,0.500088
...,...,...
4,auditory,0.000853
2,temp,0.000590
3,36,0.000590
1,emphysema,0.000482


In [95]:
# Setting Correlation threshold
top_tfidf_features_df = tfidf_output_df[:500]
filtered_tfidf_train = tfidf_train.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_test = tfidf_test.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_valid   = tfidf_valid.filter(items=top_tfidf_features_df['Features'])

In [96]:
# svm = LinearSVC(probability=True)
svm = SVC(probability=True, kernel='linear')
svm.fit(filtered_tfidf_train, y_train)
y_prob_svm = svm.predict_proba(filtered_tfidf_valid)
labels_svm = svm.predict(filtered_tfidf_valid)

In [97]:
target_names = ['NO', 'NTR', 'YES']
results_svm = classification_report(y_valid, labels_svm, target_names=target_names, output_dict=True)
results_svm = pd.DataFrame(results_svm).transpose()
print(results_svm)
print("AUC: ", roc_auc_score(y_valid, y_prob_svm, multi_class = 'ovr'))

print("\nConfusion Matrix: \n", confusion_matrix(y_valid, labels_svm))

              precision    recall  f1-score     support
NO             1.000000  1.000000  1.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.981818  1.000000  0.990826   54.000000
accuracy       0.991453  0.991453  0.991453    0.991453
macro avg      0.993939  0.994536  0.994187  117.000000
weighted avg   0.991608  0.991453  0.991457  117.000000
AUC:  1.0

Confusion Matrix: 
 [[ 2  0  0]
 [ 0 60  1]
 [ 0  0 54]]


In [99]:
for c in [0.01, 0.1, 1, 10, 100]:
    labels_lgr_tfidf = np.array([])

    lr = LogisticRegression(C=c, random_state=0)
    lr.fit(filtered_tfidf_train, y_train)
    
    y_prob_lr = lr.predict_proba(filtered_tfidf_valid)
    #print("C:", c, "\n: ", )

    labels_lgr_tfidf = lr.predict(filtered_tfidf_valid)

    results_lgr = classification_report(y_valid, labels_lgr_tfidf, target_names=target_names, output_dict=True)
    results_lgr = pd.DataFrame(results_lgr).transpose()
    print("\nC: ", c, "\n", results_svm)
    print("AUC: ", roc_auc_score(y_valid, y_prob_lr, multi_class='ovr'))
    #print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
    print("\nConfusion Matrix: \n", confusion_matrix(y_valid, labels_lgr_tfidf))
    print("---------------------------------------------------------------------\n")


C:  0.01 
               precision    recall  f1-score     support
NO             1.000000  1.000000  1.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.981818  1.000000  0.990826   54.000000
accuracy       0.991453  0.991453  0.991453    0.991453
macro avg      0.993939  0.994536  0.994187  117.000000
weighted avg   0.991608  0.991453  0.991457  117.000000
AUC:  0.9996088768098482

Confusion Matrix: 
 [[ 0  0  2]
 [ 0 60  1]
 [ 0  0 54]]
---------------------------------------------------------------------


C:  0.1 
               precision    recall  f1-score     support
NO             1.000000  1.000000  1.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.981818  1.000000  0.990826   54.000000
accuracy       0.991453  0.991453  0.991453    0.991453
macro avg      0.993939  0.994536  0.994187  117.000000
weighted avg   0.991608  0.991453  0.991457  117.000000
AUC:  0.9999024199843872

Confusion Mat

In [101]:
clf_random_forest = RandomForestClassifier()
clf_random_forest.fit(filtered_tfidf_train, y_train)

y_prob_rf = clf_random_forest.predict_proba(filtered_tfidf_valid)
    #print("C:", c, "\n: ", )

labels_rf_tfidf = clf_random_forest.predict(filtered_tfidf_valid)

results_rf = classification_report(y_valid, labels_lgr_tfidf, target_names=target_names, output_dict=True)
results_rf = pd.DataFrame(results_lgr).transpose()
print("\nC: ", c, "\n", results_rf)
print("AUC: ", roc_auc_score(y_valid, y_prob_rf, multi_class='ovr'))
#print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid, labels_lgr_tfidf))
print("---------------------------------------------------------------------\n")


NameError: name 'RandomForestClassifier' is not defined